In [1]:
import numpy as np
import pandas as pd
import math

### Imputing values for excel: Disney movies total gross

#### Imputation method:

Most of the missing values are in columns genre and rating. These cannot be calculated. So we will replace the missing values with "Unknown". We are also printing the number of unknown values at the end, so that we can keep it in mind when proceeding further.

In [3]:
#Read the total gross data csv file

total_gross_data = pd.read_csv("disney_movies_total_gross.csv")
total_gross_data = total_gross_data[['movie_title','release_date','genre','MPAA_rating','total_gross','inflation_adjusted_gross']]
print(total_gross_data.head())

   index                      movie_title  release_date      genre  \
0      0  Snow White and the Seven Dwarfs  Dec 21, 1937    Musical   
1      1                        Pinocchio   Feb 9, 1940  Adventure   
2      2                         Fantasia  Nov 13, 1940    Musical   
3      3                Song of the South  Nov 12, 1946  Adventure   
4      4                       Cinderella  Feb 15, 1950      Drama   

  MPAA_rating   total_gross inflation_adjusted_gross  
0           G  $184,925,485           $5,228,953,251  
1           G   $84,300,000           $2,188,229,052  
2           G   $83,320,000           $2,187,090,808  
3           G   $65,000,000           $1,078,510,579  
4           G   $85,000,000             $920,608,730  


In [16]:
#Replace blank values with NaN
total_gross_data_no_na = total_gross_data[['movie_title','release_date','genre','MPAA_rating','total_gross','inflation_adjusted_gross']].apply(lambda x: x.str.strip()).replace(np.nan, "Unknown")
print(total_gross_data_no_na.head(10))
print("Unknown genre", total_gross_data_no_na[total_gross_data_no_na.genre == 'Unknown'].shape[0])
print("Unknown MPAA",total_gross_data_no_na[total_gross_data_no_na.MPAA_rating == 'Unknown'].shape[0])

                       movie_title  release_date      genre MPAA_rating  \
0  Snow White and the Seven Dwarfs  Dec 21, 1937    Musical           G   
1                        Pinocchio   Feb 9, 1940  Adventure           G   
2                         Fantasia  Nov 13, 1940    Musical           G   
3                Song of the South  Nov 12, 1946  Adventure           G   
4                       Cinderella  Feb 15, 1950      Drama           G   
5     20,000 Leagues Under the Sea  Dec 23, 1954  Adventure     Unknown   
6               Lady and the Tramp  Jun 22, 1955      Drama           G   
7                  Sleeping Beauty  Jan 29, 1959      Drama     Unknown   
8                   101 Dalmatians  Jan 25, 1961     Comedy           G   
9      The Absent Minded Professor  Mar 16, 1961     Comedy     Unknown   

    total_gross inflation_adjusted_gross  
0  $184,925,485           $5,228,953,251  
1   $84,300,000           $2,188,229,052  
2   $83,320,000           $2,187,090,808  
3 

In [17]:
#Save the new csv file
total_gross_data_no_na.to_csv("cleaned_gross_data.csv")

### Imputing values for excel: Disney revenue 1991 - 2016

#### Imputation method:

Here, most of the values are present as numbers. We can see that the total is filled for all the years. This will be the key for imputation.
If one column value is missing, we can figure that out by adding the rest of the columns and subtracting from the total.
The rest of the Unknowns cannot be filled and will be taken as such.

In [48]:
disney_revenue_raw = pd.read_csv("disney_revenue_1991-2016.csv")
disney_revenue = disney_revenue_raw[['Year','Studio Entertainment[NI 1]','Disney Consumer Products[NI 2]',
                                    'Disney Interactive[NI 3][Rev 1]','Walt Disney Parks and Resorts',
                                    'Disney Media Networks','Total']].copy()
disney_revenue = disney_revenue[['Year','Studio Entertainment[NI 1]','Disney Consumer Products[NI 2]',
                                 'Disney Interactive[NI 3][Rev 1]','Walt Disney Parks and Resorts',
                                 'Disney Media Networks','Total']].apply(lambda x:x).replace(np.nan, -99)

disney_revenue["Disney Media Networks"] = disney_revenue["Disney Media Networks"].replace(",", "", regex=True)

print(disney_revenue.dtypes)

#print(disney_revenue)

Year                                 int64
Studio Entertainment[NI 1]         float64
Disney Consumer Products[NI 2]     float64
Disney Interactive[NI 3][Rev 1]    float64
Walt Disney Parks and Resorts      float64
Disney Media Networks               object
Total                                int64
dtype: object


In [59]:
for index, row in disney_revenue.iterrows():
    all_numbers = [row["Studio Entertainment[NI 1]"],row["Disney Consumer Products[NI 2]"],
                   row["Disney Interactive[NI 3][Rev 1]"],row["Walt Disney Parks and Resorts"],int(row["Disney Media Networks"])]
    total = row["Total"]
    no_of_missing = all_numbers.count(-99)
    #print(all_numbers)
    #print(no_of_missing)
    if no_of_missing == 1:
        #print(index,"Imputing")
        sum_of_nos = math.floor(sum(all_numbers))+99
        #print(all_numbers)
        #print(total,sum_of_nos)
        missing_num = total - sum_of_nos
        index_miss = all_numbers.index(-99)
        #row[index_miss+1] = missing_num
        disney_revenue.at[index,'Disney Interactive[NI 3][Rev 1]'] = missing_num
        #print("missing num",missing_num)
    
print(disney_revenue.head(4))

   Year  Studio Entertainment[NI 1]  Disney Consumer Products[NI 2]  \
0  1991                      2593.0                           724.0   
1  1992                      3115.0                          1081.0   
2  1993                      3673.4                          1415.1   
3  1994                      4793.0                          1798.2   

   Disney Interactive[NI 3][Rev 1]  Walt Disney Parks and Resorts  \
0                            -99.0                         2794.0   
1                            -99.0                         3306.0   
2                            -99.0                         3440.7   
3                              1.0                         3463.6   

  Disney Media Networks  Total    2  
0                   -99   6111  NaN  
1                   -99   7502  NaN  
2                   -99   8529  NaN  
3                   359  10414  1.0  


In [61]:
disney_revenue.to_csv("cleaned_disney_revenue.csv")

### Imputing values for excel: Disney director

#### Imputation method:
Here, the data for directors is mostly present. But to get more insights, we are combining this data with the gross data. We are combining the data on "movie name"

In [7]:
disney_director = pd.read_csv("disney-director.csv")
disney_director = disney_director[['name','director']].copy()
print(disney_director.head(3))

total_gross_data = pd.read_csv("cleaned_gross_data.csv")
total_gross_data = total_gross_data[['movie_title','release_date','genre','MPAA_rating','total_gross','inflation_adjusted_gross']]
print(total_gross_data.head(3))

                              name        director
0  Snow White and the Seven Dwarfs      David Hand
1                        Pinocchio  Ben Sharpsteen
2                         Fantasia    full credits
                       movie_title  release_date      genre MPAA_rating  \
0  Snow White and the Seven Dwarfs  Dec 21, 1937    Musical           G   
1                        Pinocchio   Feb 9, 1940  Adventure           G   
2                         Fantasia  Nov 13, 1940    Musical           G   

    total_gross inflation_adjusted_gross  
0  $184,925,485           $5,228,953,251  
1   $84,300,000           $2,188,229,052  
2   $83,320,000           $2,187,090,808  


In [8]:
for index, row in disney_director.iterrows():
    corr_revenue = total_gross_data.loc[total_gross_data['movie_title']==row["name"]]
    if not corr_revenue.empty:
        disney_director.at[index,'release_date']=corr_revenue.iloc[0]["release_date"]
        disney_director.at[index,'genre']=corr_revenue.iloc[0]["genre"]
        disney_director.at[index,'MPAA_rating']=corr_revenue.iloc[0]["MPAA_rating"]
        disney_director.at[index,'total_gross']=corr_revenue.iloc[0]["total_gross"]
        disney_director.at[index,'inflation_adjusted_gross']=corr_revenue.iloc[0]["inflation_adjusted_gross"]
    #print(row)
    
print(disney_director.head(5))

                              name        director  release_date      genre  \
0  Snow White and the Seven Dwarfs      David Hand  Dec 21, 1937    Musical   
1                        Pinocchio  Ben Sharpsteen   Feb 9, 1940  Adventure   
2                         Fantasia    full credits  Nov 13, 1940    Musical   
3                            Dumbo  Ben Sharpsteen           NaN        NaN   
4                            Bambi      David Hand           NaN        NaN   

  MPAA_rating   total_gross inflation_adjusted_gross  
0           G  $184,925,485           $5,228,953,251  
1           G   $84,300,000           $2,188,229,052  
2           G   $83,320,000           $2,187,090,808  
3         NaN           NaN                      NaN  
4         NaN           NaN                      NaN  


In [9]:
#Replace blank values with NaN
disney_director["release_date"].fillna("unknown", inplace=True)
disney_director["genre"].fillna("unknown", inplace=True)
disney_director["MPAA_rating"].fillna("unknown", inplace=True)

disney_director["total_gross"].fillna(0, inplace=True)
disney_director["inflation_adjusted_gross"].fillna(0, inplace=True)

print(disney_director.head(5))


                              name        director  release_date      genre  \
0  Snow White and the Seven Dwarfs      David Hand  Dec 21, 1937    Musical   
1                        Pinocchio  Ben Sharpsteen   Feb 9, 1940  Adventure   
2                         Fantasia    full credits  Nov 13, 1940    Musical   
3                            Dumbo  Ben Sharpsteen       unknown    unknown   
4                            Bambi      David Hand       unknown    unknown   

  MPAA_rating   total_gross inflation_adjusted_gross  
0           G  $184,925,485           $5,228,953,251  
1           G   $84,300,000           $2,188,229,052  
2           G   $83,320,000           $2,187,090,808  
3     unknown             0                        0  
4     unknown             0                        0  


In [10]:
#Check percentages of unknown
print("Unknown genre", disney_director[disney_director.genre == 'unknown'].shape[0])

Unknown genre 11


In [11]:
disney_director.to_csv("cleaned_disney_director.csv")